[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AutoViML/featurewiz_polars/blob/main/examples/featurewiz-polars-test.ipynb)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import sys
print(sys.version)
from featurewiz_polars import FeatureWiz, FeatureWiz_Model, polars_train_test_split
from sklearn.model_selection import train_test_split 


3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:20:11) [MSC v.1938 64 bit (AMD64)]
Imported featurewiz_polars 0.3.0. Use the following syntax:
 >> from featurewiz_polars import FeatureWiz, FeatureWiz_Model
    


In [3]:
#datapath = '/kaggle/input/sales-dataset/'
datapath = '../data/'
filename = 'house_price_regression.csv'

In [4]:
#!pip install git+https://github.com/AutoViML/featurewiz_polars.git

In [5]:
import polars as pl
df = pl.read_csv(datapath+filename, null_values=['NULL','NA'], try_parse_dates=True, infer_schema_length=10000, ignore_errors=True)
#df = pd.read_csv(datapath+filename)
print(df.shape)
df.head()

(545, 13)


price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
i64,i64,i64,i64,i64,str,str,str,str,str,i64,str,str
13300000,7420,4,2,3,"""yes""","""no""","""no""","""no""","""yes""",2,"""yes""","""furnished"""
12250000,8960,4,4,4,"""yes""","""no""","""no""","""no""","""yes""",3,"""no""","""furnished"""
12250000,9960,3,2,2,"""yes""","""no""","""yes""","""no""","""no""",2,"""yes""","""semi-furnished"""
12215000,7500,4,2,2,"""yes""","""no""","""yes""","""no""","""yes""",3,"""yes""","""furnished"""
11410000,7420,4,1,2,"""yes""","""yes""","""yes""","""no""","""yes""",2,"""no""","""furnished"""


# There are some outliers which are distorting the results. So we will remove them!

In [6]:
print(df.shape)
df = df.filter(pl.col("price") < 10000000) 
df.shape

(545, 13)


(537, 13)

In [7]:
model_type = "Regression"
#model_type = "Classification"
target = 'price'
predictors = [x for x in df.columns if x not in [target]]
len(predictors)

12

In [8]:
X = df[predictors] 
y = df[target] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
mrmr = FeatureWiz_Model(model_type=model_type, 
                corr_limit=0.7, category_encoders='ordinal', classic=True, 
                verbose=0)

In [12]:
X_transformed, y_transformed = mrmr.fit_transform(X_train, y_train) #np.log(y_train)


Featurewiz Polars started. Model type: Regression
SULOV selected Features (12): ['airconditioning', 'area', 'basement', 'bathrooms', 'bedrooms', 'furnishingstatus', 'guestroom', 'hotwaterheating', 'mainroad', 'parking', 'prefarea', 'stories']

Recursive XGBoost selected Features (10): ['mainroad', 'stories', 'prefarea', 'bedrooms', 'guestroom', 'furnishingstatus', 'bathrooms', 'airconditioning', 'area', 'hotwaterheating']

Featurewiz-Polars feature selection with XGBoost estimator completed.
Time taken  = 0.9 seconds

Featurewiz-Polars MRMR completed training with XGBoost Model.
Total time taken  = 0.9 seconds


In [13]:
X_transformed.head(2)

mainroad,stories,prefarea,bedrooms,guestroom,furnishingstatus,bathrooms,airconditioning,area,hotwaterheating
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.0,2.0,0.0,3.0,0.0,2.0,1.0,1.0,3000.0,1.0
1.0,2.0,0.0,3.0,0.0,2.0,1.0,0.0,4520.0,1.0


In [14]:
y_pred = mrmr.predict(X_test)
y_pred

array([2901780.        , 5042415.        , 4685800.        ,
       7195335.        , 3250170.        , 7770327.6       ,
       4214700.        , 3057110.        , 5692750.        ,
       5370820.        , 5970475.        , 4685800.        ,
       4059440.        , 7523868.8       , 4712540.        ,
       4972240.        , 3840865.        , 6877885.        ,
       6268500.        , 7841400.        , 4420010.        ,
       2850750.        , 6018390.        , 4241160.        ,
       6811210.        , 6278370.        , 3669890.        ,
       5524610.        , 4921000.        , 3656205.        ,
       5507530.        , 5488770.        , 4417980.        ,
       3790710.        , 2921409.16666667, 3213350.        ,
       4129760.83333333, 6482210.        , 4129428.33333333,
       3841158.33333333, 3573920.        , 4026505.        ,
       3072230.        , 3879890.        , 4970000.        ,
       3157980.        , 7268450.        , 3981180.        ,
       3211985.        ,

In [15]:
if model_type == 'Classification':
    y_test = mrmr.y_encoder.transform(y_test)
    print(y_test.to_numpy().ravel())
else:
    print(y_test.to_numpy().ravel())

[2660000 6615000 3773000 6300000 3080000 6510000 3143000 1890000 6300000
 4830000 9240000 4480000 3430000 6580000 4480000 2233000 2870000 6090000
 5425000 7035000 4550000 5110000 5320000 3885000 4830000 9800000 3850000
 3710000 8890000 3570000 5943000 5215000 2870000 4270000 1890000 2961000
 4200000 6629000 3780000 4340000 5495000 3710000 2310000 2940000 4193000
 6475000 5740000 4550000 3500000 7962500 4585000 2485000 3150000 8960000
 2100000 5110000 6083000 4795000 5600000 5523000 4900000 4060000 6510000
 1820000 6650000 2835000 2940000 3640000 3640000 4410000 6510000 4690000
 3118850 2660000 6650000 2653000 6755000 3087000 3220000 6440000 3780000
 7840000 6405000 6293000 9870000 8855000 3395000 8400000 3710000 3640000
 2940000 5215000 5460000 3640000 8463000 4900000 3360000 8645000 4200000
 7490000 2660000 2135000 7350000 6230000 3920000 3990000 4095000 1750000]


In [16]:
from featurewiz_polars import print_regression_metrics, print_classification_metrics
if model_type == 'Classification':
    print_classification_metrics(y_test.to_numpy().ravel(), y_pred, verbose=1)
else:
    print_regression_metrics(y_test.to_numpy().ravel(), y_pred, verbose=1)

    RMSE = 1235906.152
    Norm RMSE = 63%
    MAE = 895249.503
    WAPE = 18%, Bias = -0.2%
    MAPE = 1995%
	Hint: high MAPE: try np.log(y) instead of (y).
    R-Squared = 60%


In [17]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, mean_squared_log_error
np.sqrt(mean_squared_error(y_test, y_pred))

1235906.1521276296

In [18]:
np.std(y_test.to_numpy()).ravel()

array([1948361.56160592])